In [0]:
## 칼텍 데이터셋 직접 다운받기
# !wget http://www.vision.caltech.edu/Image_Datasets/Caltech101/101_ObjectCategories.tar.gz
# !tar -zxvf 101_ObjectCategories.tar.gz

In [0]:
! yes | pip3 uninstall opencv-python
! yes | pip3 uninstall opencv-contrib-python
! yes | pip3 install opencv-python==3.4.2.16
! yes | pip3 install opencv-contrib-python==3.4.2.16

Uninstalling opencv-python-3.4.7.28:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-3.4.7.28.dist-info/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtCore-9549151f.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libQtGui-6d0f14dd.so.4.8.7
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavcodec-eac15e48.so.58.21.104
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavformat-b6bcbe33.so.58.17.101
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libavutil-e1b1a17d.so.56.18.102
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswresample-b4363bfa.so.3.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libswscale-15b3fdc6.so.5.2.100
    /usr/local/lib/python3.6/dist-packages/cv2/.libs/libvpx-81a43c0a.so.5.0.0
Proceed (y/n)?   Successfully uninstalled opencv-python-3.4.7.28
Uninstalling opencv-contrib-python-3.4.3.

In [0]:
# 캐글 API 버전 갱신을 위한 작업... 반드시 버전이 1.5.6 이어야 함
!ls -lha kaggle.json

!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

-rw-r--r-- 1 root root 63 Dec  3 03:17 kaggle.json
Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.3.1)
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-cp36-none-any.whl size=72859 sha256=b132c547436538c59755ec96347f08d5aa1f60d7f9bddfb01017aea4c6bb4ecb
  Stored in directory: /root/.cache/pip/wheels/57/4e/e8/bb28d035162fb8f17f8ca5d42c3230e284c6aa565b42b72674
Successfully built kaggle


In [0]:
# 캐글연동을 위한 토큰 입력
! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

# 버전이 1.5.6 이 아니면, 진행할 수 없다
! kaggle -v

Kaggle API 1.5.6


In [0]:
! kaggle competitions download -c 2019-ml-finalproject
! unzip 2019-ml-finalproject.zip

 97% 63.0M/65.0M [00:00<00:00, 190MB/s]
100% 65.0M/65.0M [00:00<00:00, 218MB/s]
Archive:  2019-ml-finalproject.zip
  inflating: Label2Names.csv         
  inflating: sample_submission.csv   
  inflating: testAll_v2/image_0001.jpg  
  inflating: testAll_v2/image_0002.jpg  
  inflating: testAll_v2/image_0003.jpg  
  inflating: testAll_v2/image_0004.jpg  
  inflating: testAll_v2/image_0005.jpg  
  inflating: testAll_v2/image_0006.jpg  
  inflating: testAll_v2/image_0007.jpg  
  inflating: testAll_v2/image_0008.jpg  
  inflating: testAll_v2/image_0009.jpg  
  inflating: testAll_v2/image_0010.jpg  
  inflating: testAll_v2/image_0011.jpg  
  inflating: testAll_v2/image_0012.jpg  
  inflating: testAll_v2/image_0013.jpg  
  inflating: testAll_v2/image_0014.jpg  
  inflating: testAll_v2/image_0015.jpg  
  inflating: testAll_v2/image_0016.jpg  
  inflating: testAll_v2/image_0017.jpg  
  inflating: testAll_v2/image_0018.jpg  
  inflating: testAll_v2/image_0019.jpg  
  inflating: testAll_v2/image_

In [0]:
pip install kmc2

     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 163kB 8.6MB/s 
  Created wheel for kmc2: filename=kmc2-0.1-cp36-cp36m-linux_x86_64.whl size=252215 sha256=3791f35df840e173e381d3d44e52477c3c7fdd0531d49b8d2b8ddf440d48b3cf
  Stored in directory: /root/.cache/pip/wheels/5c/ba/f0/4c8b421be72d4f2d1a93233c2f6f591e7d8b0bda05a1f4616f
Successfully built kmc2


In [0]:
import os
import cv2
import kmc2
import numpy as np
import pandas as pd
import cupy as xp

from glob import glob
from PIL import Image
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC,SVC
from scipy.cluster.vq import vq
from sklearn.metrics import confusion_matrix
from sklearn import svm

In [0]:
code_book_size = 600
img_size = 256
Step_size = 8
cls_num = 101
des_size = int(img_size/Step_size)
p1 = int(des_size*(1/4))
p2 = int(des_size*(2/4))
p3 = int(des_size*(3/4))

In [0]:
df_data = pd.read_csv('/content/Label2Names.csv', header=None)

In [0]:
# sift 선언
sift = cv2.xfeatures2d.SIFT_create()
scaler = StandardScaler()

In [0]:
DATA_ROOT_TRAIN = "./train"
train_des = list()
train_images = list()
train_labels = list()

for cls in tqdm(os.listdir(DATA_ROOT_TRAIN)):
  img_list = os.listdir(DATA_ROOT_TRAIN+'/'+cls)
  img_list.sort()

  if cls == 'BACKGROUND_Google' :
    label = 102
    if cls_num == 101 :
      continue
  else :
    label = (df_data.index[df_data[1]==cls]+1).tolist()[0]

  for ii,img in enumerate(img_list) :

    image = cv2.imread(DATA_ROOT_TRAIN+'/'+cls+'/'+img)
    gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (img_size,img_size))

    kp = [cv2.KeyPoint(x, y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1], Step_size)]
    keypoint, des = sift.compute(gray, kp)

    train_des.append(des)
    train_images.append(gray)
    train_labels.append(label)

train_labels = np.array(train_labels)

In [0]:
np.array(train_des).reshape(-1,128).shape

(3102720, 128)

In [0]:
seeding = kmc2.kmc2(np.array(train_des).reshape(-1,128), code_book_size)
Kmeas = MiniBatchKMeans(code_book_size, init=seeding).fit(np.array(train_des).reshape(-1,128))
codebook = Kmeas.cluster_centers_

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: Explicit initial center position passed: performing only one init in MiniBatchKMeans instead of n_init=3
  
/usr/local/lib/python3.6/dist-packages/sklearn/cluster/k_means_.py:1561: RuntimeWarning: init_size=300 should be larger than k=600. Setting it to 3*k
  init_size=init_size)


In [0]:
total_train_feature = list()

for i in tqdm(range(len(train_des))) :

  ## level 0
  level_0 = vq(train_des[i],codebook)
  level_0 = np.histogram(level_0[0],bins=list(range(code_book_size+1)))[0]

  des_map = train_des[i].reshape(des_size,des_size,128)

  ## level 1
  level_1_1 = vq(des_map[:p2,:p2,:].reshape(-1,128),codebook)
  level_1_1_train_feature = np.histogram(level_1_1[0],bins=list(range(code_book_size+1)))
  level_1_1_train_feature = level_1_1_train_feature[0].reshape(code_book_size,1)

  level_1_2 = vq(des_map[p2:,:p2,:].reshape(-1,128),codebook)
  level_1_2_train_feature = np.histogram(level_1_2[0],bins=list(range(code_book_size+1)))
  level_1_2_train_feature = level_1_2_train_feature[0].reshape(code_book_size,1)

  level_1_3 = vq(des_map[:p2,p2:,:].reshape(-1,128),codebook)
  level_1_3_train_feature = np.histogram(level_1_3[0],bins=list(range(code_book_size+1)))
  level_1_3_train_feature = level_1_3_train_feature[0].reshape(code_book_size,1)

  level_1_4 = vq(des_map[p2:,p2:,:].reshape(-1,128),codebook)
  level_1_4_train_feature = np.histogram(level_1_4[0],bins=list(range(code_book_size+1)))
  level_1_4_train_feature = level_1_4_train_feature[0].reshape(code_book_size,1)

  level_1 = np.concatenate((level_1_1_train_feature,level_1_2_train_feature,level_1_3_train_feature,level_1_4_train_feature),axis=1).flatten()

  ## level 2
  level_2_1 = vq(des_map[:p1,:p1,:].reshape(-1,128),codebook)
  level_2_1_train_feature = np.histogram(level_2_1[0],bins=list(range(code_book_size+1)))
  level_2_1_train_feature = level_2_1_train_feature[0].reshape(code_book_size,1)

  level_2_2 = vq(des_map[p1:p2,:p1,:].reshape(-1,128),codebook)
  level_2_2_train_feature = np.histogram(level_2_2[0],bins=list(range(code_book_size+1)))
  level_2_2_train_feature = level_2_2_train_feature[0].reshape(code_book_size,1)

  level_2_3 = vq(des_map[p2:p3,p1:,:].reshape(-1,128),codebook)
  level_2_3_train_feature = np.histogram(level_2_3[0],bins=list(range(code_book_size+1)))
  level_2_3_train_feature = level_2_3_train_feature[0].reshape(code_book_size,1)

  level_2_4 = vq(des_map[p3:,p1:,:].reshape(-1,128),codebook)
  level_2_4_train_feature = np.histogram(level_2_4[0],bins=list(range(code_book_size+1)))
  level_2_4_train_feature = level_2_4_train_feature[0].reshape(code_book_size,1)

  level_2_5 = vq(des_map[:p1,p1:p2,:].reshape(-1,128),codebook)
  level_2_5_train_feature = np.histogram(level_2_5[0],bins=list(range(code_book_size+1)))
  level_2_5_train_feature = level_2_5_train_feature[0].reshape(code_book_size,1)

  level_2_6 = vq(des_map[p1:p2,p1:p2,:].reshape(-1,128),codebook)
  level_2_6_train_feature = np.histogram(level_2_6[0],bins=list(range(code_book_size+1)))
  level_2_6_train_feature = level_2_6_train_feature[0].reshape(code_book_size,1)

  level_2_7 = vq(des_map[p2:p3,p1:p2,:].reshape(-1,128),codebook)
  level_2_7_train_feature = np.histogram(level_2_7[0],bins=list(range(code_book_size+1)))
  level_2_7_train_feature = level_2_7_train_feature[0].reshape(code_book_size,1)

  level_2_8 = vq(des_map[p3:,p1:p2,:].reshape(-1,128),codebook)
  level_2_8_train_feature = np.histogram(level_2_8[0],bins=list(range(code_book_size+1)))
  level_2_8_train_feature = level_2_8_train_feature[0].reshape(code_book_size,1)

  level_2_9 = vq(des_map[:p1,p2:p3,:].reshape(-1,128),codebook)
  level_2_9_train_feature = np.histogram(level_2_9[0],bins=list(range(code_book_size+1)))
  level_2_9_train_feature = level_2_9_train_feature[0].reshape(code_book_size,1)

  level_2_10 = vq(des_map[p1:p2,p2:p3,:].reshape(-1,128),codebook)
  level_2_10_train_feature = np.histogram(level_2_10[0],bins=list(range(code_book_size+1)))
  level_2_10_train_feature = level_2_10_train_feature[0].reshape(code_book_size,1)

  level_2_11 = vq(des_map[p2:p3,p2:p3,:].reshape(-1,128),codebook)
  level_2_11_train_feature = np.histogram(level_2_11[0],bins=list(range(code_book_size+1)))
  level_2_11_train_feature = level_2_11_train_feature[0].reshape(code_book_size,1)

  level_2_12 = vq(des_map[p3:,p2:p3,:].reshape(-1,128),codebook)
  level_2_12_train_feature = np.histogram(level_2_12[0],bins=list(range(code_book_size+1)))
  level_2_12_train_feature = level_2_12_train_feature[0].reshape(code_book_size,1)

  level_2_13 = vq(des_map[:p1,p3:,:].reshape(-1,128),codebook)
  level_2_13_train_feature = np.histogram(level_2_13[0],bins=list(range(code_book_size+1)))
  level_2_13_train_feature = level_2_13_train_feature[0].reshape(code_book_size,1)

  level_2_14 = vq(des_map[p1:p2,p3:,:].reshape(-1,128),codebook)
  level_2_14_train_feature = np.histogram(level_2_14[0],bins=list(range(code_book_size+1)))
  level_2_14_train_feature = level_2_14_train_feature[0].reshape(code_book_size,1)

  level_2_15 = vq(des_map[p2:p3,p3:,:].reshape(-1,128),codebook)
  level_2_15_train_feature = np.histogram(level_2_15[0],bins=list(range(code_book_size+1)))
  level_2_15_train_feature = level_2_15_train_feature[0].reshape(code_book_size,1)

  level_2_16 = vq(des_map[p3:,p3:,:].reshape(-1,128),codebook)
  level_2_16_train_feature = np.histogram(level_2_16[0],bins=list(range(code_book_size+1)))
  level_2_16_train_feature = level_2_16_train_feature[0].reshape(code_book_size,1)

  level_2 = np.concatenate((level_2_1_train_feature,level_2_2_train_feature,level_2_3_train_feature,level_2_4_train_feature,
                            level_2_5_train_feature,level_2_6_train_feature,level_2_7_train_feature,level_2_8_train_feature,
                            level_2_9_train_feature,level_2_10_train_feature,level_2_11_train_feature,level_2_12_train_feature,
                            level_2_13_train_feature,level_2_14_train_feature,level_2_15_train_feature,level_2_16_train_feature),axis=1).flatten()

  total_train_feature.append(np.concatenate((0.25*level_0,0.25*level_1,0.5*level_2)))
  # total_train_feature.append(level_0)

total_train_feature = np.array(total_train_feature)

In [0]:
scaler.fit(total_train_feature)
total_train_feature = scaler.transform(total_train_feature)

In [0]:
DATA_ROOT_TEST = "./testAll_v2"

test_des = list()
test_images = list()

img_list = os.listdir(DATA_ROOT_TEST)
img_list.sort()

for img in tqdm(img_list):
  image = cv2.imread(DATA_ROOT_TEST+'/'+img)
  gray= cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  gray = cv2.resize(gray, (img_size,img_size))   

  kp = [cv2.KeyPoint(x, y, Step_size) for y in range(0, gray.shape[0], Step_size) for x in range(0, gray.shape[1], Step_size)]
  keypoint, des = sift.compute(gray, kp)

  test_des.append(des)
  test_images.append(gray)

In [0]:
total_test_feature = list()

for i in tqdm(range(len(test_des))) :

  ## level 0
  level_0 = vq(test_des[i],codebook)
  level_0 = np.histogram(level_0[0],bins=list(range(code_book_size+1)))[0]

  des_map = test_des[i].reshape(des_size,des_size,128)

  ## level 1
  level_1_1 = vq(des_map[:p2,:p2,:].reshape(-1,128),codebook)
  level_1_1_test_feature = np.histogram(level_1_1[0],bins=list(range(code_book_size+1)))
  level_1_1_test_feature = level_1_1_test_feature[0].reshape(code_book_size,1)

  level_1_2 = vq(des_map[p2:,:p2,:].reshape(-1,128),codebook)
  level_1_2_test_feature = np.histogram(level_1_2[0],bins=list(range(code_book_size+1)))
  level_1_2_test_feature = level_1_2_test_feature[0].reshape(code_book_size,1)

  level_1_3 = vq(des_map[:p2,p2:,:].reshape(-1,128),codebook)
  level_1_3_test_feature = np.histogram(level_1_3[0],bins=list(range(code_book_size+1)))
  level_1_3_test_feature = level_1_3_test_feature[0].reshape(code_book_size,1)

  level_1_4 = vq(des_map[p2:,p2:,:].reshape(-1,128),codebook)
  level_1_4_test_feature = np.histogram(level_1_4[0],bins=list(range(code_book_size+1)))
  level_1_4_test_feature = level_1_4_test_feature[0].reshape(code_book_size,1)

  level_1 = np.concatenate((level_1_1_test_feature,level_1_2_test_feature,level_1_3_test_feature,level_1_4_test_feature),axis=1).flatten()

  ## level 2
  level_2_1 = vq(des_map[:p1,:p1,:].reshape(-1,128),codebook)
  level_2_1_test_feature = np.histogram(level_2_1[0],bins=list(range(code_book_size+1)))
  level_2_1_test_feature = level_2_1_test_feature[0].reshape(code_book_size,1)

  level_2_2 = vq(des_map[p1:p2,:p1,:].reshape(-1,128),codebook)
  level_2_2_test_feature = np.histogram(level_2_2[0],bins=list(range(code_book_size+1)))
  level_2_2_test_feature = level_2_2_test_feature[0].reshape(code_book_size,1)

  level_2_3 = vq(des_map[p2:p3,p1:,:].reshape(-1,128),codebook)
  level_2_3_test_feature = np.histogram(level_2_3[0],bins=list(range(code_book_size+1)))
  level_2_3_test_feature = level_2_3_test_feature[0].reshape(code_book_size,1)

  level_2_4 = vq(des_map[p3:,p1:,:].reshape(-1,128),codebook)
  level_2_4_test_feature = np.histogram(level_2_4[0],bins=list(range(code_book_size+1)))
  level_2_4_test_feature = level_2_4_test_feature[0].reshape(code_book_size,1)

  level_2_5 = vq(des_map[:p1,p1:p2,:].reshape(-1,128),codebook)
  level_2_5_test_feature = np.histogram(level_2_5[0],bins=list(range(code_book_size+1)))
  level_2_5_test_feature = level_2_5_test_feature[0].reshape(code_book_size,1)

  level_2_6 = vq(des_map[p1:p2,p1:p2,:].reshape(-1,128),codebook)
  level_2_6_test_feature = np.histogram(level_2_6[0],bins=list(range(code_book_size+1)))
  level_2_6_test_feature = level_2_6_test_feature[0].reshape(code_book_size,1)

  level_2_7 = vq(des_map[p2:p3,p1:p2,:].reshape(-1,128),codebook)
  level_2_7_test_feature = np.histogram(level_2_7[0],bins=list(range(code_book_size+1)))
  level_2_7_test_feature = level_2_7_test_feature[0].reshape(code_book_size,1)

  level_2_8 = vq(des_map[p3:,p1:p2,:].reshape(-1,128),codebook)
  level_2_8_test_feature = np.histogram(level_2_8[0],bins=list(range(code_book_size+1)))
  level_2_8_test_feature = level_2_8_test_feature[0].reshape(code_book_size,1)

  level_2_9 = vq(des_map[:p1,p2:p3,:].reshape(-1,128),codebook)
  level_2_9_test_feature = np.histogram(level_2_9[0],bins=list(range(code_book_size+1)))
  level_2_9_test_feature = level_2_9_test_feature[0].reshape(code_book_size,1)

  level_2_10 = vq(des_map[p1:p2,p2:p3,:].reshape(-1,128),codebook)
  level_2_10_test_feature = np.histogram(level_2_10[0],bins=list(range(code_book_size+1)))
  level_2_10_test_feature = level_2_10_test_feature[0].reshape(code_book_size,1)

  level_2_11 = vq(des_map[p2:p3,p2:p3,:].reshape(-1,128),codebook)
  level_2_11_test_feature = np.histogram(level_2_11[0],bins=list(range(code_book_size+1)))
  level_2_11_test_feature = level_2_11_test_feature[0].reshape(code_book_size,1)

  level_2_12 = vq(des_map[p3:,p2:p3,:].reshape(-1,128),codebook)
  level_2_12_test_feature = np.histogram(level_2_12[0],bins=list(range(code_book_size+1)))
  level_2_12_test_feature = level_2_12_test_feature[0].reshape(code_book_size,1)

  level_2_13 = vq(des_map[:p1,p3:,:].reshape(-1,128),codebook)
  level_2_13_test_feature = np.histogram(level_2_13[0],bins=list(range(code_book_size+1)))
  level_2_13_test_feature = level_2_13_test_feature[0].reshape(code_book_size,1)

  level_2_14 = vq(des_map[p1:p2,p3:,:].reshape(-1,128),codebook)
  level_2_14_test_feature = np.histogram(level_2_14[0],bins=list(range(code_book_size+1)))
  level_2_14_test_feature = level_2_14_test_feature[0].reshape(code_book_size,1)

  level_2_15 = vq(des_map[p2:p3,p3:,:].reshape(-1,128),codebook)
  level_2_15_test_feature = np.histogram(level_2_15[0],bins=list(range(code_book_size+1)))
  level_2_15_test_feature = level_2_15_test_feature[0].reshape(code_book_size,1)

  level_2_16 = vq(des_map[p3:,p3:,:].reshape(-1,128),codebook)
  level_2_16_test_feature = np.histogram(level_2_16[0],bins=list(range(code_book_size+1)))
  level_2_16_test_feature = level_2_16_test_feature[0].reshape(code_book_size,1)

  level_2 = np.concatenate((level_2_1_test_feature,level_2_2_test_feature,level_2_3_test_feature,level_2_4_test_feature,
                            level_2_5_test_feature,level_2_6_test_feature,level_2_7_test_feature,level_2_8_test_feature,
                            level_2_9_test_feature,level_2_10_test_feature,level_2_11_test_feature,level_2_12_test_feature,
                            level_2_13_test_feature,level_2_14_test_feature,level_2_15_test_feature,level_2_16_test_feature),axis=1).flatten()

  total_test_feature.append(np.concatenate((0.25*level_0,0.25*level_1,0.5*level_2)))
  # total_test_feature.append(level_0)

total_test_feature = np.array(total_test_feature)

In [0]:
total_test_feature = scaler.transform(total_test_feature)

In [0]:
def histogramIntersection(M, N):
    m = M.shape[0]
    n = N.shape[0]
    result = np.zeros((m,n))
    for i in range(m):
        for j in range(n):
            temp = np.sum(np.minimum(M[i], N[j]))
            result[i][j] = temp
    return result

In [0]:
gramMatrix = histogramIntersection(total_train_feature, total_train_feature)
clf = svm.SVC(kernel='precomputed')
clf.fit(gramMatrix, train_labels)
predictMatrix = histogramIntersection(total_test_feature, total_train_feature)
SVMResults = clf.predict(predictMatrix)

In [0]:
SVMResults = SVMResults.reshape(-1,1)
result_img_list = np.array(img_list).reshape(-1,1)
total_result = np.hstack([result_img_list,SVMResults])

In [0]:
df = pd.DataFrame(total_result, columns=["Id","Category"])
df.to_csv('results-jwkim-v3-precompute-imgsize256-step8-c600-scaler.csv',index=False, header=True)

In [0]:
! kaggle competitions submit -c 2019-ml-finalproject -f results-jwkim-v3-precompute-imgsize256-step8-c600-scaler.csv -m "Final_Term_Project"

100% 29.6k/29.6k [00:00<00:00, 115kB/s]
Successfully submitted to 2019.Fall.PatternRecognition 